# Giải thích file `drl/learning/rher.py`

Tài liệu này tổng hợp các thành phần chính của `drl/learning/rher.py` (phiên bản trong workspace).

## Mục đích
- `RHER` là một framework huấn luyện dùng Hindsight Experience Replay cho bài toán nhiều giai đoạn (multi-stage). Lớp `RHER` kế thừa `RLFrameWork` và quản lý luồng: tạo rollout, lưu vào bộ nhớ (memory), huấn luyện agent và đánh giá.

## Các import chính
- `torch`, `numpy`, `tqdm`, `SummaryWriter` — thư viện ML, xử lý tensor và ghi log TensorBoard.
- Từ dự án: `RLFrameWork`, `map_structure`, `put_structure`, `groupby_structure`, `nearest_node_value`, `MeanMetrics`, `LOGGER`, `format_time`, `format_tabulate` — các tiện ích và kiểu cơ sở.

## Kiến trúc lớp `RHER`
- `__init__(envs, agent, memory, compute_metrics=None)`
  - Kiểm tra interface của `agent` và `memory` (ví dụ `memory.reward_func`, `memory.num_stages`, `memory.horizon`).
  - `compute_metrics` mặc định: kết hợp `goal_achieved` và tỉ lệ reward/horizon để tạo `eval_value` tổng hợp.

### Exploration / stage handling
- `_get_stages(observations, mt_goals)`
  - Gọi `memory.reward_func` cho từng `stage_id` để nhận `goal_achieved` tại mọi stage.
  - Tính `stages` hiện tại cho mỗi phần tử trong batch bằng cách xét stage lớn nhất đã đạt.

- `_select_action(observations, mt_goals, stages, *, r_mix=0.5)`
  - `r_mix` (0..1) là xác suất để "trộn" goal: với xác suất `r_mix` mẫu sẽ được chuyển từ stage hiện tại sang stage kế tiếp trước khi chọn action.
  - Mục đích: hướng dẫn exploration theo mục tiêu (goal-directed exploration) bằng cách đôi khi cung cấp goal của stage tiếp theo.
  - Gọi `agent({'observation':..., 'goal':..., 'task_id': stages}, deterministic=False)` để lấy action non-deterministic.

- `select_actions(..., deterministic=False)`
  - Nếu không deterministic: chuẩn hoá dữ liệu bằng `agent.format_data`, gọi `_get_stages` và `_select_action`, trả về `(actions_numpy, goal_achieveds_bool)`.
  - Nếu deterministic: dùng goal cuối cùng và stage cuối, gọi policy deterministic.

### Training
- `train(num_updates, batch_size, future_p=0.8, n_steps=1, step_decay=0.7, discounted_factor=0.99, clip_return=None)`
  - Nếu dùng prioritized replay thì cập nhật priorities trước.
  - Lặp `num_updates` lần: sample từ `memory.sample(...)` (ở đây HER sampling được xử lý trong memory), gọi `agent.update(...)` để cập nhật mạng.
  - Trả về `MeanMetrics` chứa các metric trung bình.

### Evaluate
- `evaluate(num_episodes=10)`
  - Chạy envs ở chế độ `eval`, chọn action deterministic, thu thông tin episode (reward, horizon, infos), sử dụng `compute_metrics` để tính `eval_value`.

### Run (luồng chính)
- `run(...)`
  - Tạo thư mục experiment (`make_exp_dir`), có thể `resume` từ checkpoint.
  - Với mỗi epoch: lặp cycles, mỗi cycle gọi `generate_rollouts`, `store_rollouts`, sau đó `train` nhiều lần.
  - Sau epoch chạy `evaluate` và lưu policy/ckpt nếu cần.

### Rollout / Memory
- `generate_rollouts(r_mix='auto')`
  - Tạo rollout theo `memory.horizon`. Mỗi step: gọi `select_actions` (non-deterministic), thực hiện `envs.step`, thu observations, action, và `goal_achieved`.
  - Trả về batch rollouts (cấu trúc đã swap axis) và `info` (mean eps_reward, eps_horizon, ...).

- `store_rollouts(rollouts)`
  - Chuẩn hoá các chiều mảng (`meta`, `achieved_goal`, `desired_goal`) trước khi gọi `memory.store(rollouts)`.

## Về `r_mix` và entropy
  - `r_mix`: exploration theo goal (thay đổi input mục tiêu để agent dẫn hướng tới stage tiếp theo).
  - Entropy: exploration ở action-space (policy thử nhiều action khác nhau cho cùng goal).

## Ví dụ cụ thể về `r_mix` và entropy trong huấn luyện cánh tay robot

### Bối cảnh mẫu
- Task: pick-and-place chia thành 4 stage: `reach` → `grasp` → `lift` → `place`.
- `memory.num_stages = 4`, mỗi rollout có `memory.horizon` bước.
- Agent có thể là SAC (policy Gaussian, có hệ số entropy `alpha`) hoặc DDPG/TD3 (thêm noise với std).

### Ý tưởng chính (tóm tắt)
- `r_mix`: xác suất tại mỗi bước để thay goal hiện tại bằng goal của stage tiếp theo trước khi policy chọn action (goal-guided exploration).
- Entropy / noise: điều chỉnh mức độ ngẫu nhiên của policy khi chọn action cho cùng một (state, goal) (action-space exploration).

### Cấu hình mẫu và hành vi mong đợi
- Cấu hình A — Thận trọng:
  - `r_mix = 0.0`, entropy thấp / noise nhỏ (SAC `alpha ≈ 0.01`, DDPG noise std ≈ 0.02).
  - Agent học chắc từng stage, ít thử nghiệm goal kế tiếp, ổn định nhưng có thể chậm qua stage.

- Cấu hình B — Cân bằng:
  - `r_mix = 0.5`, entropy moderate (SAC `alpha ≈ 0.05 - 0.2`, DDPG std ≈ 0.05 - 0.15).
  - Một phần rollout hướng tới mục tiêu tiếp theo, policy vẫn thử nhiều hành động — có lợi cho chuyển tiếp giữa stage.

- Cấu hình C — Hướng mạnh mục tiêu tiếp theo:
  - `r_mix = 1.0`, entropy thấp (nếu entropy cao đồng thời có thể gây bất ổn).
  - Agent liên tục nhận goal của stage tiếp theo, đẩy nhanh học chuyển giai đoạn nhưng có rủi ro nếu kỹ năng cơ bản yếu.

- Thử nghiệm khám phá rộng:
  - `r_mix = 0.8` kết hợp entropy cao (SAC `alpha` lớn hoặc noise std cao) → khám phá rất rộng, khó hội tụ nhưng có thể tìm chuỗi hành động phức tạp.

### Ví dụ số cho pick-and-place
- Setup: `batch_size=256`, `horizon=50`, `num_stages=4`.
- Grid thử: `r_mix` ∈ {0.0, 0.25, 0.5, 0.75, 1.0} × SAC `alpha` ∈ {0.01, 0.05, 0.2} (hoặc DDPG std ∈ {0.02, 0.08, 0.2}).
- Theo dõi: success rate per stage, time-to-first-success, average reward, variance. Chạy nhiều seed để kiểm nghiệm.

### Cách cấu hình trong code
- Truyền `r_mix` khi gọi `run()`:

```python
# ví dụ
rher.run(..., r_mix=0.5, num_updates=50, batch_size=256)
```

- Cấu hình entropy cho SAC (ví dụ):

```python
# nếu agent là SAC
sac_agent = SACAgent(..., entropy_coef=0.05)
```

- Cấu hình noise cho DDPG:

```python
ddpg_agent = DDPGAgent(..., action_noise_std=0.08)
```

### Lưu ý tuning & thử nghiệm đề xuất
- Bắt đầu với `r_mix=0.25-0.5` và entropy moderate (SAC alpha auto hoặc ~0.05). Quan sát: nếu agent không tiến qua stage 1→2 thì tăng `r_mix`; nếu agent thất bại nhiều ở stage hiện tại thì giảm `r_mix` hoặc giảm entropy.
- Thử một biến tại một thời điểm (chỉ đổi `r_mix` hoặc chỉ đổi entropy`) để thấy tác động rõ ràng.
- Thử nghiệm đề xuất (ngắn):
  1. Baseline: `r_mix=0.0`, SAC `alpha=0.05`.
  2. Thêm `r_mix=0.5`, giữ `alpha` cố định — so sánh success per stage.
  3. Giữ `r_mix=0.5`, thay đổi `alpha` lên/xuống.

### Cảnh báo
- `generate_rollouts` trong repo có default `r_mix='auto'` nhưng `_select_action` mong `r_mix` là số trong `[0,1]`. Hãy đảm bảo khi gọi `generate_rollouts()` truyền giá trị số thực hoặc sửa `generate_rollouts` để xử lý `'auto'` hợp lý.

## Lưu ý thực dụng
- `memory.reward_func` phải hỗ trợ `stage_id` (multi-stage reward).
- `generate_rollouts` mặc định `r_mix='auto'` nhưng `_select_action` cần `r_mix` là số trong `[0,1]`. Nếu gọi `generate_rollouts()` mà không truyền `r_mix` thì `'auto'` có thể gây lỗi. Nên sửa để xử lý `'auto'` hoặc luôn truyền `r_mix` từ `run()`.
- `agent` phải cung cấp `format_data`, `device`, `update`, và chấp nhận input `{'observation','goal','task_id'}`.
- `memory` phải implement `store`, `sample`, `reward_func`, `num_stages`, `horizon`, và nếu dùng prioritized replay thì `use_priority` + `update_priorities()`.

## 📚 Giải thích chi tiết về Reuse Ratio

### **1. Reuse Ratio là gì?**

**Reuse Ratio** (Tỷ lệ tái sử dụng) là số lần trung bình mỗi sample trong replay buffer được sử dụng lại cho training trong một khoảng thời gian (thường là 1 epoch).

**Công thức:**


In [ ]:
Reuse Ratio = Total Samples Consumed / New Samples Collected



### **2. Tính toán cho cấu hình của bạn:**

#### **Thu thập dữ liệu (Data Collection):**


In [ ]:
num_envs = 32
num_cycles = 800  
# → Số episodes thu thập mỗi epoch = 800 episodes

horizon = 128 steps/episode
# → Transitions mới mỗi epoch = 800 × 128 = 102,400 transitions



#### **Tiêu thụ dữ liệu (Data Consumption):**


In [ ]:
batch_size = 512
num_updates = 256 per cycle
cycles_per_epoch = 800 / 32 = 25 batches

# Tổng updates mỗi epoch:
total_updates = 256 × 25 = 6,400 updates

# Tổng samples tiêu thụ mỗi epoch:
total_samples = 512 × 6,400 = 3,276,800 samples



#### **Reuse Ratio:**


In [ ]:
Reuse Ratio = 3,276,800 / 102,400 ≈ 32×



**Chính xác hơn với RHER:**
- RHER sample từ multiple stages (num_stages=2)
- Mỗi stage sample riêng: `3,276,800 / 2 = 1,638,400 samples/stage`
- Nhưng vẫn từ cùng pool 102,400 transitions
- **Effective Reuse Ratio ≈ 32×** (không phải 4096× như tôi tính nhầm trước!)

---

### **3. Ý nghĩa của Reuse Ratio:**

#### **🔄 Reuse Ratio = 1× (Ideal On-Policy)**


In [ ]:
Collection:  100 samples
Consumption: 100 samples (sử dụng đúng 1 lần)

- **Ví dụ**: PPO, TRPO (on-policy algorithms)
- Mỗi sample chỉ dùng 1 lần rồi bỏ
- Sample efficiency thấp nhưng tránh overfitting

#### **🔄 Reuse Ratio = 10-50× (Normal Off-Policy)**


In [ ]:
Collection:  100 samples/epoch
Consumption: 1,000-5,000 samples/epoch
Reuse:       10-50×

- **Ví dụ**: SAC, DDPG với replay buffer tiêu chuẩn
- Sample nhiều lần từ buffer lớn
- Cân bằng giữa sample efficiency và stability

#### **🔄 Reuse Ratio = 100-1000× (High Reuse)**


In [ ]:
Collection:  100 samples/epoch  
Consumption: 10,000-100,000 samples/epoch
Reuse:       100-1000×

- Thường gặp khi:
  - Buffer lớn, collect ít
  - Nhiều updates per cycle
  - Multi-stage sampling (như RHER)
- **Risk**: Overfitting to replay buffer

#### **🚨 Reuse Ratio = 4000× (Extreme Overfitting)**


In [ ]:
Collection:  100 samples
Consumption: 400,000 samples

- **Dấu hiệu**: Model học "thuộc lòng" buffer
- Chất lượng policy không cải thiện
- Training loss giảm nhưng eval performance không tăng

---

### **4. Cấu hình của bạn: Reuse = 32×**

#### **Tái tính toán chính xác:**

**Mỗi epoch:**


In [ ]:
# Data collection:
new_transitions = 800 episodes × 128 steps = 102,400 transitions

# Data consumption:
samples_per_update = 512 (batch_size)
updates_per_epoch = 256 updates × 25 cycles = 6,400 updates
total_samples = 512 × 6,400 = 3,276,800 samples

# Reuse ratio:
reuse = 3,276,800 / 102,400 = 32×



#### **✅ Đánh giá: MODERATE - ACCEPTABLE**

**32× reuse là mức độ cao nhưng vẫn chấp nhận được cho off-policy RL:**

| Metric | Value | Status |
|--------|-------|--------|
| Reuse Ratio | **32×** | 🟡 Moderate-High |
| Typical SAC | 10-50× | ✅ In range |
| Risk Level | Medium | ⚠️ Monitor needed |

---

### **5. Tại sao Reuse cao KHÔNG phải lúc nào cũng xấu?**

#### **🎯 Ưu điểm của High Reuse (với off-policy RL):**

**A. Sample Efficiency cao hơn:**


In [ ]:
On-policy (PPO):     1M samples → 1M training steps
Off-policy (SAC):    100K samples → 3.2M training steps (32× reuse)
                     → Học nhanh hơn 32× với cùng lượng sim data!



**B. Ổn định hơn với diverse buffer:**


In [ ]:
max_length = 20,000 trajectories
→ Pool size = 20,000 × 128 = 2.56M transitions

# Random sampling từ 2.56M transitions
# → Mỗi update thấy data khác nhau
# → Giảm overfitting dù reuse cao



**C. HER augmentation:**


In [ ]:
future_p = 0.8
→ 80% samples được relabel với hindsight goals
→ Tăng diversity hiệu dụng
→ Effective reuse thấp hơn nominal



#### **⚠️ Rủi ro của High Reuse:**

**A. Q-value overestimation:**


In [ ]:
Critic học quá fit với buffer
→ Q-values biased
→ Policy exploitation sai



**B. Slow adaptation to new data:**


In [ ]:
Model "ám ảnh" với old data
→ Học chậm từ new experience
→ Plateau performance



**C. Overfitting symptoms:**


In [ ]:
Training loss ↓↓↓ (giảm rất nhanh)
Eval performance ↑↓↑ (không ổn định)
Train-eval gap ↑↑↑ (chênh lệch lớn)



---

### **6. Monitoring & Debugging Reuse Issues:**

#### **📊 Metrics cần theo dõi:**

**A. Training metrics:**


In [ ]:
# Trong TensorBoard/logs:
- actor_loss: Nên giảm đều
- critic_loss: Nên giảm đều, không spike
- approx_ent: Entropy không nên drop quá nhanh
- ent_coef: Alpha tự điều chỉnh, quan sát trend



**B. Evaluation metrics:**


In [ ]:
- eps_reward: Phải tăng theo epochs
- goal_achieved: Success rate tăng
- distance: Error giảm dần
- eval_value: Q-estimate nên tương quan với reward



**C. Buffer metrics:**


In [ ]:
# Check trong logs:
- Buffer length: Tăng dần đến max_length
- Sampling distribution: Uniform vs priority



#### **🔍 Dấu hiệu overfitting:**



In [ ]:
# BAD patterns:
epoch 10: train_loss=0.05, eval_reward=150
epoch 20: train_loss=0.01, eval_reward=160  # Loss giảm nhanh
epoch 30: train_loss=0.005, eval_reward=155 # Reward không tăng/giảm
epoch 40: train_loss=0.001, eval_reward=140 # Reward giảm!

# GOOD patterns:
epoch 10: train_loss=0.05, eval_reward=150
epoch 20: train_loss=0.03, eval_reward=180  # Cả 2 cải thiện
epoch 30: train_loss=0.02, eval_reward=210
epoch 40: train_loss=0.015, eval_reward=235



---

### **7. Giải pháp điều chỉnh Reuse Ratio:**

#### **🎛️ Option 1: Giảm Reuse (tăng collection)**



In [ ]:
# Tăng số cycles = tăng episodes collected
learner.run(
    num_cycles=1600,  # 800 → 1600
    num_updates=256,
    batch_size=512,
)
# → Reuse = 3.28M / 204,800 = 16× (giảm một nửa)



**Pros:** Giảm overfitting, diverse data
**Cons:** Chậm hơn, tốn simulation time

#### **🎛️ Option 2: Giảm Reuse (giảm updates)**



In [ ]:
learner.run(
    num_cycles=800,
    num_updates=128,  # 256 → 128
    batch_size=512,
)
# → Reuse = 1.64M / 102,400 = 16×



**Pros:** Nhanh hơn Option 1, ít overfitting
**Cons:** Học chậm hơn, cần nhiều epochs

#### **🎛️ Option 3: Giảm Reuse (giảm batch size)**



In [ ]:
learner.run(
    num_cycles=800,
    num_updates=256,
    batch_size=256,  # 512 → 256
)
# → Reuse = 1.64M / 102,400 = 16×



**Pros:** Giảm memory, đơn giản
**Cons:** Gradient noise cao hơn, ít stable

#### **🎛️ Option 4: Tăng Buffer (giảm % reuse)**



In [ ]:
memory = RHERMemory(
    max_length=40000,  # 20,000 → 40,000
    ...
)
# Reuse vẫn 32×, nhưng sample từ pool lớn gấp đôi
# → Diversity tăng, overfitting giảm



**Pros:** Diversity cao, không ảnh hưởng training speed
**Cons:** Tốn VRAM (~3-4GB thay vì ~2GB)

---

### **8. Khuyến nghị cho config của bạn:**

#### **✅ GIỮ NGUYÊN (32× reuse) nếu:**
- Training loss và eval reward đều cải thiện đều đặn
- Success rate tăng theo epochs
- Không thấy dấu hiệu overfitting (train-eval gap không tăng)
- Muốn training nhanh

#### **⚠️ ĐIỀU CHỈNH nếu thấy:**

**Scenario A: Performance plateau sau 50-100 epochs**


In [ ]:
# → Giảm updates/cycle
num_updates=128  # thay vì 256



**Scenario B: Train loss giảm nhưng eval không cải thiện**


In [ ]:
# → Tăng collection
num_cycles=1600  # thay vì 800



**Scenario C: VRAM còn dư (>4GB free)**


In [ ]:
# → Tăng buffer size
max_length=40000  # thay vì 20000



---

## 🎯 **Tóm tắt:**

| Aspect | Your Config | Assessment |
|--------|-------------|------------|
| **Reuse Ratio** | **32×** | 🟡 Moderate-High |
| **Collection/epoch** | 102,400 transitions | ✅ Good |
| **Consumption/epoch** | 3.28M samples | 🟡 High but ok |
| **Buffer size** | 20,000 traj (2.56M) | ✅ Sufficient |
| **Risk level** | Medium | ⚠️ Monitor needed |
| **Action** | **Start with current, monitor, adjust if needed** | 👍 |

**Reuse 32× là mức hợp lý cho SAC + RHER. Không cần điều chỉnh ngay, nhưng theo dõi training curves để phát hiện overfitting sớm!** 📈

## Giải thích 'rollout' (đơn giản và cụ thể)

- Rollout = chuỗi các tương tác (transitions) thu được khi chạy policy trong môi trường: observations, actions, rewards, next-observations, và thông tin liên quan (done, infos, achieved/desire goals...).
- Ở repo này, `generate_rollouts` tạo rollout có độ dài `memory.horizon` (còn `envs.num_envs` là số môi trường song song).

### Shapes & lý do cắt dữ liệu
- Với `num_envs = N`, `horizon = H`, sau `generate_rollouts` ta có (ví dụ):
  - `rollouts['observation']`: shape `(N, H+1, ...)` — bao gồm observation ban đầu và H lần next-observation.
  - `rollouts['action']`: shape `(N, H, ...)` — hành động tại mỗi step.
  - `rollouts['achieved_goal']`: shape `(N, H+1, ...)`.
  - `rollouts['desired_goal']`: thường thu được `(N, H+1, ...)` nhưng khi store sẽ cắt thành `(N, H, ...)` tương ứng với actions.

- Tại sao `store_rollouts` cắt/move các mảng:
  - Khi lưu transitions cần khớp `(state_t, action_t, reward_t, next_state_{t+1})`.
  - Do đó `achieved_goal` được cắt `[:, 1:]` (bỏ achieved_goal lúc time=0) để khớp với `action_t` → chính là achieved ở next state.
  - `desired_goal` được cắt `[:, :-1]` (bỏ goal cuối) để mỗi `action_t` có `desired_goal` tương ứng ở thời điểm t.
  - Nếu có `meta`, cũng cắt bỏ phần đầu để khớp tương tự.

### Vai trò của rollout trong HER / RHER
- Rollout là nguồn dữ liệu thô; HER thay nhãn (relabel) goals trong memory khi sample (đổi desired goal thành achieved goal ở một thời điểm sau trong cùng rollout) để học từ trải nghiệm "hindsight".
- Ở RHER, `r_mix` có tác động ngay khi tạo rollout: một số step sẽ dùng goal của stage tiếp theo khi chọn action, vì vậy rollout phản ánh cả việc trộn goal và exploration hướng mục tiêu.

### Ví dụ ngắn (pick-and-place)
- Nếu `N=8`, `H=50`: `generate_rollouts` thu `8×51` observations, `8×50` actions, `8×51` achieved_goal. Sau `store_rollouts` lưu vào memory các mảng dạng `8×50` cho transitions `(state_t, action_t, next_state, desired_goal_t, achieved_goal_next, goal_achieved_flag)`.
